# Http

## 1.1. Http server

In [ ]:
function input() {
    return new Promise((resolve, reject) => {
        const stdin = process.stdin;
        stdin.setDefaultEncoding('utf-8');
    });
}

### 1.1.1. Response result

In [ ]:
class Result {
    constructor(code) {
        this._code = code;
        this._encoding = 'utf-8';
        this._contentType = 'text/html';
        this._body = null;
        this._html = '';
    }
    
    get code() {
        return this._code;
    }
    
    get encoding() {
        return this._encoding;
    }
    
    get contentType() {
        return this._contentType;
    }
    
    get body() {
        return this._body;
    }
    
    get html() {
        return this._html;
    }
    
    static ok() {
        return new Result(200);
    }
    
    static badRequest() {
        return new Result(400);
    }
    
    json() {
        this._contentType = 'application/json';
        return this;
    }
    
    withEncoding(encoding) {
        this._encoding = encoding;
        return this;
    }
    
    withBody(obj) {
        this._body = obj;
        return this;
    }
    
    withHtml(html) {
        this._html = html;
        return this;
    }
}

#### 1.1.1. Create server

In [ ]:
const http = require('http');
const url = require('url');
const qs = require('querystring');

class HttpServer {
    constructor(port, router, encoding='UTF-8', bindAddress='0.0.0.0') {
        this._server = this._createServer(router);
        this._server.listen(port, bindAddress);
    }
    
    _createServer(router, encoding) {
        router = router || {};
        return http.createServer((req, resp) => {
            const href = url.parse(req.url);
            console.log(`* request comming, path is "${href.pathname}"`);
            
            const callback = router[href.pathname];
            if (callback) {
                console.log(`* find route of this request`);
                this._receiveRequest(req, resp, href, encoding, callback);
            } else {
                this._sendErrorResponse(resp, 404, encoding);
            }
        });
    }
    
    _sendErrorResponse(resp, code, encoding='utf-8') {
        resp.writeHead(code, {
            'Content-Type': `text/html; charset=${encoding}`,
            'Content-Length': 0
        });
        resp.end();
    }
    
    _receiveRequest(req, resp, href, encoding, callback) {
        const chunks = [];
        
        resp.on('finish', () => console.log('* response sent'));
        
        req.setEncoding('utf-8');
        req.on('data', chunk => chunks.push(chunk));
        req.on('end', () => {
            const req = {
                pathname: href.pathname,
                parameters: {},
                querystring: qs.parse(href.query),
                body: ''
            };
            if (href.query) {
                Object.assign(req.parameters, qs.parse(href.query));
            } else {
                req.body = chunks.join('');
            }

            const result = callback(req, resp);
            encoding = result.encoding || encoding;
            
            const content = result.raw || '';
            const headers = Object.assign({
                    'Content-Type': `${result.contentType}; charset=${encoding}`,
                    'Content-Length': Buffer.byteLength(content, encoding)
                }, result.headers || {});
            
            let body = '';
            switch (result.contentType) {
                case 'text/html':
                    body = result.html;
                    break;
                case 'application/json':
                    body = JSON.stringify(result.body || null);
                    break;
            }
            resp.write(body, encoding);
            resp.end('');
        });
    }
}

#### 1.1.2. Testing

- Client request

In [ ]:
const https = require('https');
const http = require('http');
const zlib = require('zlib');
const url = require('url');

function request(uri, method, data, contentType, headers, encoding) {
    return new Promise((resolve, reject) => {
        const options = Object.assign({}, url.parse(uri));
        /** @type {any} */
        let proto = http;
        if (options.protocol === 'https:') {
            proto = https;
            Object.assign(options, {
                rejectUnauthorized: false,
                requestCert: true
            });
        }
        
        if (data) {
            Object.assign(headers, {
                'Content-Type': `${contentType}; charset=${encoding}`,
                "Content-Length": Buffer.byteLength(data, encoding)
            });
        }
        Object.assign(options, {
            headers: Object.assign({
                'Accept-Encoding': 'gzip,deflate',
            }, headers)
        });
        
        const req = proto.request(options, resp => {
            switch (resp.headers['content-encoding']) {
            case 'gzip':
                (unzip => {
                    resp.pipe(unzip);
                    resp = unzip;
                })(zlib.createGunzip());
                break;
            case 'deflate':
                (deflate => {
                    resp.pipe(deflate);
                    resp = deflate;
                })(zlib.createDeflate());
            }
            
            const chunks = [];
            resp.on('data', chunk => {
                chunks.push(Buffer.from(chunk));
            });

            resp.on('end', () => {
                const buffer = Buffer.concat(chunks);
                resolve({
                    headers: resp.headers,
                    code: resp.statusCode,
                    data: buffer.toString(encoding)
                });
            });
            
            resp.on('error', err => reject(err));
        });
        
        req.on('error', err => reject(err));
        
        if (data) {
            req.write(data, encoding);
        }
        req.end();
    });
}

async function get(url, headers=null, encoding='utf-8') {
    return await request(url, 'GET', null, null, headers || {}, encoding);
}

async function post(url, data, contentType='application/x-www-form-urlencoded', headers=null, encoding='utf-8') {
    return await request(url, 'POST', null, null, headers || {}, encoding);
}

- Start server

In [ ]:
{
    const server = new HttpServer(9090, {
        '/': req => {
            return Result.ok().withHtml(`<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Hello</title>
    <style type="text/css">
        div.main {
            width: 1000px;
            margin: 0 auto;
        }
        nav>div {
            text-align: right;
        }
        nav a {
            font-size: 12px;
        }
    </style>
</head>
<body>
    <div class="main">
        <header>
            <nav>
                <div><a href="/d/version">Version</a></div>
            </nav>
        </header>
        <main>
            <section>
                <h2 align="center">Hello World</h2>
            </section>
        </main>
    </div>
</body>
</html>`);
        },
        '/d/version': req => {
            const version = {version: '1.0.0', build: 101};
            return Result.ok().json().withBody(version);
        }
    });    
}

- Send request and get response

In [ ]:
{
    let resp = await get('http://localhost:9090');
    console.log(`* get "/" return:`);
    console.log(resp);
    
    resp = await get('http://localhost:9090/d/version');
    console.log(`* get "/d/version" return:`);
    console.log(resp);
}